In [1]:
%cd ../

/Users/origenolet/Desktop/Ironhack/Week9/Final_bootcamp_project


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import functions.functions as f
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE 
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, f1_score, cohen_kappa_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import pickle

In [3]:
data = pd.read_csv('./data/raw/hypothyroid.csv')
display(data.head())
print(data.shape)

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
0,41,F,f,f,f,f,f,f,f,f,...,t,125,t,1.14,t,109,f,?,SVHC,P
1,23,F,f,f,f,f,f,f,f,f,...,t,102,f,?,f,?,f,?,other,P
2,46,M,f,f,f,f,f,f,f,f,...,t,109,t,0.91,t,120,f,?,other,P
3,70,F,t,f,f,f,f,f,f,f,...,t,175,f,?,f,?,f,?,other,P
4,70,F,f,f,f,f,f,f,f,f,...,t,61,t,0.87,t,70,f,?,SVI,P


(3772, 30)


In [4]:
data = f.data_cleaning(data)

In [5]:
data = f.feature_selection(data)
data.head()

,sex,on_thyroxine,on_antithyroid_medication,pregnant,goitre,tsh,t3,fti,binaryclass
0,F,f,f,f,f,1.30,2.5,109.0,Healthy
1,F,f,f,f,f,4.10,2.0,107.0,Healthy
2,M,f,f,f,f,0.98,2.0,120.0,Healthy
3,F,t,f,f,f,0.16,1.9,107.0,Healthy
4,F,f,f,f,f,0.72,1.2,70.0,Healthy


In [6]:
data.to_csv("./data/clean/hypothyroid_clean.csv", index = False)

In [7]:
data[data["binaryclass"] == "Diseased"]

,sex,on_thyroxine,on_antithyroid_medication,pregnant,goitre,tsh,t3,fti,binaryclass
15,F,f,f,f,f,12.0,2.0,87.0,Diseased
40,M,f,f,f,f,45.0,1.4,33.0,Diseased
57,M,f,f,f,f,14.8,1.5,72.0,Diseased
58,F,f,f,f,f,15.0,1.6,100.0,Diseased
59,F,f,f,f,f,19.0,2.2,81.0,Diseased
...,...,...,...,...,...,...,...,...,...
3740,F,f,f,f,f,117.0,0.4,15.0,Diseased
3748,M,f,f,f,f,55.0,0.6,17.0,Diseased
3751,F,f,f,f,f,6.3,1.1,65.0,Diseased
3753,F,f,f,f,f,40.0,2.2,74.0,Diseased


### Enconding features ### 

In [ ]:
### Sex column

In [ ]:
data["sex"] = data["sex"].replace({"F":1, "M":0})

In [ ]:
### Other categoricals with f, t

cols = ["on_thyroxine", "on_antithyroid_medication", "pregnant", "goitre"]
for col in cols:
    data[col] = data[col].replace({"t":1, "f":0})

In [ ]:
data.head()

### X, y split ###

In [ ]:
X = data.drop("binaryclass", axis = 1)
y = data["binaryclass"]

### Train test split ### 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Shape X train:', X_train.shape)
print('Shape X test:', X_test.shape)
print('Shape y train:', y_train.shape)
print('Shape y test:', y_test.shape)

### MinMax scaler ### 

In [ ]:
scaler_num = MinMaxScaler().fit(X_train)
X_train_scaled = scaler_num.transform(X_train)

filename = "./transformers/minmaxscaler.pickle" 
with open(filename, "wb") as file:
    pickle.dump(scaler_num,file)
       
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns, index=X_train.index)
print(X_train_scaled.shape)
X_train_scaled.head()

In [ ]:
X_test_scaled = scaler_num.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns, index=X_test.index)
print(X_test_scaled.shape)
X_test_scaled.head()

### Class imbalance: SMOTE ###

In [ ]:
sm = SMOTE(random_state=50, k_neighbors=3)
X_train_scaled_sm, y_train_sm = sm.fit_resample(X_train_scaled, y_train)
print('X shape before using SMOTE:', X_train_scaled.shape)
print('X shape after using SMOTE:', X_train_scaled_sm.shape)
print('')
print('Y value counts:\n', y_train_sm.value_counts())

In [ ]:
X_train_scaled_sm.head()

In [ ]:
y_train_sm.head()

### Class imbalance: Upsampling ###

In [ ]:
### generating train set again to upsample the donor samples

trainset = pd.concat([X_train_scaled, y_train], axis=1)
print(trainset.shape)
trainset.head()

In [ ]:
### creating healthy and diseased dataset for upsampling

healthy = trainset[trainset["binaryclass"] == "Healthy"]
diseased = trainset[trainset["binaryclass"] == "Diseased"]
print(healthy.shape)
print(diseased.shape)

In [ ]:
### upsampling donors dataset

diseased_upsample = resample(diseased,
             replace=True,
             n_samples=len(healthy),
             random_state=42)

print(diseased_upsample.shape)
display(diseased_upsample.head())

In [ ]:
### here I am concatenating both datasets again and doing the x, y split for the train set again

trainset_new = pd.concat([diseased_upsample, healthy], axis = 0)
trainset_new = trainset_new.sample(frac =1).reset_index(drop=True)
X_train_scaled_up = trainset_new.drop(['binaryclass'], axis=1)
y_train_up = trainset_new['binaryclass']

In [ ]:
X_train_scaled_up.head()

In [ ]:
y_train_up.head()

### Logistic regression ###

**Best parameter search with bagging** 

In [ ]:
LR = LogisticRegression()
clf_lr = BaggingClassifier(base_estimator=LR, random_state=50)
scorer = make_scorer(f1_score, pos_label = "Diseased")

n_estimators = [5,10,15]
solvers = ["lbfgs", "liblinear"]

grid_lr = {
        'base_estimator__solver': solvers,
        'n_estimators': n_estimators
        }

In [ ]:
grid_search = GridSearchCV(estimator = clf_lr, 
                           scoring = scorer, 
                           param_grid = grid_lr, 
                           cv = 5, 
                           verbose = 2)

In [ ]:
lr_grid = grid_search.fit(X_train_scaled_sm, y_train_sm)

In [ ]:
print('Best parameters:', lr_grid.best_params_)

**Logistic regression model with bagging (SMOTE)**

In [ ]:
LR = LogisticRegression(solver = "liblinear")
clf_lr = BaggingClassifier(base_estimator=LR, random_state=50, n_estimators = 5)
clf_lr.fit(X_train_scaled_sm, y_train_sm)

In [ ]:
y_train_pred = clf_lr.predict(X_train_scaled_sm)
y_test_pred  = clf_lr.predict(X_test_scaled)

In [ ]:
cm_train = confusion_matrix(y_train_sm, y_train_pred)
disp = ConfusionMatrixDisplay(cm_train,display_labels=clf_lr.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(cm_test,display_labels=clf_lr.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
print("The f1 score in the TRAIN set is: {:.3f}".format(f1_score(y_train_sm, y_train_pred, pos_label="Diseased")))
print("The f1 score in the TEST  set is: {:.3f}".format(f1_score(y_test, y_test_pred, pos_label="Diseased")))

**Logistic regression model with bagging (Upsampling)**

In [ ]:
LR = LogisticRegression(solver = "liblinear")
clf_lr = BaggingClassifier(base_estimator=LR, random_state=50, n_estimators = 5)
clf_lr.fit(X_train_scaled_up, y_train_up)

In [ ]:
y_train_pred = clf_lr.predict(X_train_scaled_up)
y_test_pred  = clf_lr.predict(X_test_scaled)

In [ ]:
cm_train = confusion_matrix(y_train_up, y_train_pred)
disp = ConfusionMatrixDisplay(cm_train,display_labels=clf_lr.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(cm_test,display_labels=clf_lr.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
print("The f1 score in the TRAIN set is: {:.3f}".format(f1_score(y_train_up, y_train_pred, pos_label="Diseased")))
print("The f1 score in the TEST  set is: {:.3f}".format(f1_score(y_test, y_test_pred, pos_label="Diseased")))

### K nearest neighbor ###

In [ ]:
error_rate = []
for k in range(1,40):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled_sm, y_train_sm)
    pred_k = knn.predict(X_test_scaled)
    error_rates = 1 - accuracy_score(pred_k,y_test)
    error_rate.append(error_rates)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(range(1,40),error_rate,color="black", linestyle="dashed", marker="o",
 markerfacecolor="black", markersize=4)
plt.title("Error Rate vs. K Value")
plt.xlabel("K")
plt.ylabel("Error Rate")

In [ ]:
KNN = KNeighborsClassifier()
clf_knn = BaggingClassifier(base_estimator=KNN, random_state=50)
scorer = make_scorer(f1_score, pos_label = "Diseased")

n_neighbors = [3,4,5,6]
weights = ['uniform', 'distance']
p = [1,2] 
n_estimators = [5,10]

grid_knn = {'base_estimator__n_neighbors': n_neighbors,
           'base_estimator__weights': weights,
           'base_estimator__p': p,
           'n_estimators': n_estimators}

In [ ]:
grid_search = GridSearchCV(estimator = clf_knn, 
                           scoring = scorer, 
                           param_grid = grid_knn, 
                           cv = 5, 
                           verbose = 2)

In [ ]:
knn_grid = grid_search.fit(X_train_scaled_sm, y_train_sm)

In [ ]:
print('Best parameters:', knn_grid.best_params_)

**KNN with bagging (SMOTE)**

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 3, p = 1, weights = "distance")
clf_knn = BaggingClassifier(base_estimator=KNN, random_state=50, n_estimators = 10)
clf_knn.fit(X_train_scaled_sm, y_train_sm)

In [ ]:
y_train_pred = clf_knn.predict(X_train_scaled_sm)
y_test_pred  = clf_knn.predict(X_test_scaled)

In [ ]:
cm_train = confusion_matrix(y_train_sm, y_train_pred)
disp = ConfusionMatrixDisplay(cm_train,display_labels=clf_knn.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(cm_test,display_labels=clf_knn.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
print("The f1 score in the TRAIN set is: {:.3f}".format(f1_score(y_train_sm, y_train_pred, pos_label="Diseased")))
print("The f1 score in the TEST  set is: {:.3f}".format(f1_score(y_test, y_test_pred, pos_label="Diseased")))

**KNN with bagging (Upscaling)**

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 3, p = 1, weights = "distance")
clf_knn = BaggingClassifier(base_estimator=KNN, random_state=50, n_estimators = 10)
clf_knn.fit(X_train_scaled_up, y_train_up)

In [ ]:
y_train_pred = clf_knn.predict(X_train_scaled_up)
y_test_pred  = clf_knn.predict(X_test_scaled)

In [ ]:
cm_train = confusion_matrix(y_train_up, y_train_pred)
disp = ConfusionMatrixDisplay(cm_train,display_labels=clf_knn.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(cm_test,display_labels=clf_knn.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
print("The f1 score in the TRAIN set is: {:.3f}".format(f1_score(y_train_up, y_train_pred, pos_label="Diseased")))
print("The f1 score in the TEST  set is: {:.3f}".format(f1_score(y_test, y_test_pred, pos_label="Diseased")))

### Naive bayes ###

In [ ]:
NB = GaussianNB()
clf_nb = BaggingClassifier(base_estimator=NB, random_state=50)
scorer = make_scorer(f1_score, pos_label = "Diseased")

n_estimators = [5,10, 15]

grid_nb = {'n_estimators': n_estimators}

In [ ]:
grid_search = GridSearchCV(estimator = clf_nb, 
                           scoring = scorer, 
                           param_grid = grid_nb, 
                           cv = 5, 
                           verbose = 2)

In [ ]:
nb_grid = grid_search.fit(X_train_scaled_sm, y_train_sm)

In [ ]:
print('Best parameters:', nb_grid.best_params_)

**Naive bayes with bagging (SMOTE)**

In [ ]:
NB = GaussianNB()
clf_nb = BaggingClassifier(base_estimator=NB, random_state=50, n_estimators =15)
clf_nb.fit(X_train_scaled_sm, y_train_sm)

In [ ]:
y_train_pred = clf_nb.predict(X_train_scaled_sm)
y_test_pred  = clf_nb.predict(X_test_scaled)

In [ ]:
cm_train = confusion_matrix(y_train_sm, y_train_pred)
disp = ConfusionMatrixDisplay(cm_train,display_labels=clf_nb.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(cm_test,display_labels=clf_nb.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
print("The f1 score in the TRAIN set is: {:.3f}".format(f1_score(y_train_sm, y_train_pred, pos_label="Diseased")))
print("The f1 score in the TEST  set is: {:.3f}".format(f1_score(y_test, y_test_pred, pos_label="Diseased")))

**KNN with bagging (Upscaling)**

In [ ]:
NB = GaussianNB()
clf_nb = BaggingClassifier(base_estimator=NB, random_state=50, n_estimators =15)
clf_nb.fit(X_train_scaled_up, y_train_up)

In [ ]:
y_train_pred = clf_nb.predict(X_train_scaled_up)
y_test_pred  = clf_nb.predict(X_test_scaled)

In [ ]:
cm_train = confusion_matrix(y_train_up, y_train_pred)
disp = ConfusionMatrixDisplay(cm_train,display_labels=clf_nb.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(cm_test,display_labels=clf_nb.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
print("The f1 score in the TRAIN set is: {:.3f}".format(f1_score(y_train_up, y_train_pred, pos_label="Diseased")))
print("The f1 score in the TEST  set is: {:.3f}".format(f1_score(y_test, y_test_pred, pos_label="Diseased")))

### Random Forest ###

In [ ]:
scorer = make_scorer(f1_score, pos_label = "Diseased")

n_estimators = [50,100, 150]
max_depth_choices= [3,5,10,15]
criterion_choices =  ["gini", "entropy"] 
min_samples_split_choices = [2,3,5] 
min_samples_leaf_choices = [1,2,3] 
max_features_choices = ["sqrt", "log2"]

grid = {'n_estimators': n_estimators,
        'max_depth': max_depth_choices,
        'criterion': criterion_choices,
        'min_samples_split': min_samples_split_choices, 
        'min_samples_leaf': min_samples_leaf_choices,
        'max_features': max_features_choices}

In [ ]:
rf_clf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf_clf, 
                           scoring = scorer, 
                           param_grid = grid, 
                           cv = 5, 
                           verbose = 2)

In [ ]:
rf_grid = grid_search.fit(X_train_scaled_sm, y_train_sm)

In [ ]:
print('Best parameters:', rf_grid.best_params_)

**Random Forest (SMOTE)**

In [ ]:
rf_clf = RandomForestClassifier(criterion = "entropy", max_depth = 10, max_features = "log2")

In [ ]:
rf_clf.fit(X_train_scaled_sm, y_train_sm)

In [ ]:
y_train_pred = rf_clf.predict(X_train_scaled_sm)
y_test_pred  = rf_clf.predict(X_test_scaled)

In [ ]:
filename = "./models/rf_clf" +".pickle"
with open(filename, "wb") as file:
    pickle.dump(rf_clf,file)

In [ ]:
cm_train = confusion_matrix(y_train_sm, y_train_pred)
disp = ConfusionMatrixDisplay(cm_train,display_labels=rf_clf.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(cm_test,display_labels=rf_clf.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
print("The f1 score in the TRAIN set is: {:.3f}".format(f1_score(y_train_sm, y_train_pred, pos_label="Diseased")))
print("The f1 score in the TEST  set is: {:.3f}".format(f1_score(y_test, y_test_pred, pos_label="Diseased")))

**Random Forest (Upsampling)**

In [ ]:
rf_clf = RandomForestClassifier(criterion = "entropy", max_depth = 10, max_features = "log2")

In [ ]:
rf_clf.fit(X_train_scaled_up, y_train_up)

In [ ]:
y_train_pred = rf_clf.predict(X_train_scaled_up)
y_test_pred  = rf_clf.predict(X_test_scaled)

In [ ]:
cm_train = confusion_matrix(y_train_up, y_train_pred)
disp = ConfusionMatrixDisplay(cm_train,display_labels=rf_clf.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
cm_test = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(cm_test,display_labels=rf_clf.classes_);
disp.plot()
fig = disp.ax_.get_figure() 
fig.set_figwidth(3)
fig.set_figheight(3)

In [ ]:
print("The f1 score in the TRAIN set is: {:.3f}".format(f1_score(y_train_up, y_train_pred, pos_label="Diseased")))
print("The f1 score in the TEST  set is: {:.3f}".format(f1_score(y_test, y_test_pred, pos_label="Diseased")))

### Final functions ### 

In [ ]:
def pre_processing(df):
    df = df.copy()
    df["sex"] = df["sex"].replace({"F":1, "M":0})
    cols = ["on_thyroxine", "on_antithyroid_medication", "pregnant", "goitre"]
    for col in cols:
        df[col] = df[col].replace({"t":1, "f":0})
    scaler = load("./transformer/mimaxscaler.pickle")
    df = scaler.transform(df)
    return df 

In [ ]:
def pre_processing_inputs(df):
    df = df.copy()
    df["sex"] = df["sex"].replace({"f":1, "m":0})
    cols = ["on_thyroxine", "on_antithyroid_medication", "pregnant", "goitre"]
    for col in cols:
        df[col] = df[col].replace({"y":1, "n":0})
    scaler = load("./transformer/mimaxscaler.pickle")
    df = scaler.transform(df)
    return df 

In [ ]:
def random_forest(df):
    df = df.copy()
    rf_clf = load("./models/rf_clf.pickle")
    pred = rf_clf.predict(df)
    return df